In [ ]:
import numpy as np, pandas as pd, joblib, pathlib
from catboost import CatBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import KFold
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

RNG = 42
N_FOLDS = 10
pathlib.Path("predictions").mkdir(exist_ok=True)

# load feature matrices
# UMich train features
df_um_tr = pd.read_csv("./output/umich_train_features_clean.csv")
X_um_tr = df_um_tr.drop(columns=["Dataset","Mixture 1","Mixture 2", "Experimental Values"]).values

# Belfaction features
X_bf_tr    = np.load("./output/belfaction_pair_train_X.npy")


# load target values
y_train = np.load("./output/belfaction_pair_train_y.npy").astype(np.float32)

print(f"UMich train shape : {X_um_tr.shape}")
print(f"Belfact train shape : {X_bf_tr.shape}")

# build base model
# UMich CatBoost
cat_params = dict(
    iterations = 400,
    depth = 6,
    learning_rate = 0.03,
    l2_leaf_reg = 3,
    loss_function = "RMSE",
    random_seed = RNG,
    verbose = 0
)

def fresh_umich():
    return CatBoostRegressor(**cat_params)

# Belfaction ExtraTrees
def fresh_belf():
    return make_pipeline(
        SimpleImputer(strategy="mean"),
        ExtraTreesRegressor(
            n_estimators = 1000,
            random_state = RNG,
            n_jobs       = -1
        )
    )

# OOF predictions for both base models
print("\ncomputing OOF predictions")

kf = KFold(N_FOLDS, shuffle=True, random_state=RNG)
oof_umich = np.empty_like(y_train, dtype=np.float32)
oof_belf = np.empty_like(y_train, dtype=np.float32)

for fold,(tr,vl) in enumerate(kf.split(X_um_tr),1):
    um = fresh_umich()
    um.fit(X_um_tr[tr], y_train[tr])
    oof_umich[vl] = um.predict(X_um_tr[vl])

    bf = fresh_belf()
    bf.fit(X_bf_tr[tr], y_train[tr])
    oof_belf[vl] = bf.predict(X_bf_tr[vl])

    print(f"fold {fold:2d} done")

np.save("predictions/oof_umich.npy", oof_umich)
np.save("predictions/oof_belf.npy", oof_belf )

# simple 50-50 ensemble
simple_blend = 0.5*oof_umich + 0.5*oof_belf
print("\nSimple 50-50 ensemble")
print("Pearson =", pearsonr(y_train, simple_blend)[0],
      "|  RMSE", np.sqrt(((y_train-simple_blend)**2).mean()))

# optimal scalar weight ensemble
from scipy.stats import pearsonr
import numpy as np

print("\nsearching best scalar weight …")

best_w, best_p, best_rmse = None, -9.0, 99.0

for w in np.linspace(0, 1, 51):
    z = (1 - w) * oof_umich + w * oof_belf
    pear = pearsonr(y_train, z)[0]
    rmse = np.sqrt(((y_train - z) ** 2).mean())

    if pear > best_p:
        best_w, best_p, best_rmse = w, pear, rmse

    print(f"w={w} | Pearson {pear} | RMSE {rmse}")

print("\n>>> best weight (max Pearson)")
print(f"w = {best_w} → Pearson {best_p} | RMSE {best_rmse}")

# Stacking meta-learner
print("\ntraining meta-learner …")
X_meta = np.column_stack([oof_umich, oof_belf])
alphas = np.logspace(-4,4,25)

meta = LinearRegression(fit_intercept=True, positive=True)
meta.fit(X_meta, y_train)
meta_pred  = meta.predict(X_meta)
print("Stack Pearson", pearsonr(y_train, meta_pred)[0],
      "| RMSE", np.sqrt(((y_train-meta_pred)**2).mean()))
joblib.dump(meta, "models/meta_model.joblib")




UMich train shape : (500, 138)
Belfact train shape : (500, 14221)

computing OOF predictions
fold  1 done
fold  2 done
fold  3 done
fold  4 done
fold  5 done
fold  6 done
fold  7 done
fold  8 done
fold  9 done
fold 10 done

–– Simple ½-½ blend ––
Pearson = 0.6455245596903486 |  RMSE 0.120729625

searching best scalar weight …
w=0.0 | Pearson 0.5659977933592109 | RMSE 0.12912186980247498
w=0.02 | Pearson 0.5702159618705038 | RMSE 0.12869641184806824
w=0.04 | Pearson 0.5743637898065502 | RMSE 0.1282779425382614
w=0.06 | Pearson 0.5784386866430715 | RMSE 0.1278664916753769
w=0.08 | Pearson 0.5824380328770289 | RMSE 0.12746216356754303
w=0.1 | Pearson 0.5863593677120997 | RMSE 0.1270650178194046
w=0.12 | Pearson 0.5902002742963645 | RMSE 0.12667511403560638
w=0.14 | Pearson 0.5939584573462474 | RMSE 0.12629252672195435
w=0.16 | Pearson 0.5976316694369294 | RMSE 0.12591731548309326
w=0.18 | Pearson 0.601217831031718 | RMSE 0.1255495548248291
w=0.2 | Pearson 0.6047149522324333 | RMSE 0.12518

['models/meta_model.joblib']

In [ ]:
## I did not use the leaderboard and test sets in the end, 
## but you can uncomment this section to use them if needed.

#X_um_lead = pd.read_csv("./umich_leaderboard_features.csv")\
                #.drop(columns=["Dataset","Mixture 1","Mixture 2"]).values
#X_um_test = pd.read_csv("./umich_test_features.csv")\
                #.drop(columns=["Dataset","Mixture 1","Mixture 2"]).values

#X_bf_lead  = np.load("./data/belfaction_pair_lead_X.npy")
#X_bf_test  = np.load("./data/belfaction_pair_test_X.npy")


## fit base models on all data & emit leaderboard / test

#cat_full = fresh_umich().fit(X_um_tr, y_train)
#cat_full.save_model("models/umich_catboost_final.cbm")
#joblib.dump(cat_full, "models/umich_catboost_final.joblib")

#belf_full = fresh_belf().fit(X_bf_tr, y_train)
#joblib.dump(belf_full, "models/belf_extratrees_final.joblib")

## predictions
#def emit(name, preds):
    #np.save(f"predictions/{name}.npy", preds)

## Leaderboard
#p_um_lead  = cat_full .predict(X_um_lead)
#p_bf_lead  = belf_full.predict(X_bf_lead)

#emit("lead_simple50", 0.5*p_um_lead + 0.5*p_bf_lead)
#emit("lead_optw"    , (1-best_w)*p_um_lead + best_w*p_bf_lead)
#emit("lead_stack"   , meta.predict(np.column_stack([p_um_lead, p_bf_lead])))

## Test
#p_um_test = cat_full .predict(X_um_test)
#p_bf_test = belf_full.predict(X_bf_test)

#emit("test_simple50", 0.5*p_um_test + 0.5*p_bf_test)
#emit("test_optw"    , (1-best_w)*p_um_test + best_w*p_bf_test)
#emit("test_stack"   , meta.predict(np.column_stack([p_um_test, p_bf_test])))
